In [0]:
# ============================================
# SILVER LAYER - LIMPEZA E VALIDAÇÃO
# ============================================
# Objetivo: Limpar, validar e enriquecer dados
# ============================================

from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

SOURCE_TABLE = "finance_bronze.transacoes_bronze"
TARGET_TABLE = "finance_silver.transacoes_silver"

print(" SILVER LAYER - Processamento de Dados")
print(f" Origem: {SOURCE_TABLE}")
print(f" Destino: {TARGET_TABLE}")

In [0]:
# ============================================
# 1. CARREGAR DADOS BRONZE
# ============================================

df_bronze = spark.read.table(SOURCE_TABLE)

print(f" Linhas carregadas do Bronze: {df_bronze.count()}")

print("\n Verificando qualidade dos dados antes da limpeza:")

print("\n NULLs por coluna:")
for col in ["data", "descricao", "valor", "tipo", "categoria"]:
    null_count = df_bronze.filter(F.col(col).isNull()).count()
    print(f"   {col}: {null_count} NULLs")

duplicates = df_bronze.groupBy("data", "descricao", "valor").count() \
    .filter(F.col("count") > 1) \
    .count()
print(f"\n Registros duplicados: {duplicates}")

invalid_values = df_bronze.filter(F.col("valor") <= 0).count()
print(f"  Valores <= 0: {invalid_values}")

In [0]:
# ============================================
# 2. LIMPEZA DE DADOS
# ============================================

print("Inicio limpeza dados: \n")

linhas_antes = df_bronze.count()
print(f"Linhas ANTES da limpeza: {linhas_antes}")

print("\n 1️. Removendo null em campos críticos")
df_clean = df_bronze.filter(
    F.col("data").isNotNull() &
    F.col("valor").isNotNull() &
    F.col("tipo").isNotNull() &
    F.col("categoria").isNotNull()
)
removidos_null = linhas_antes - df_clean.count()
print(f" Removidos {removidos_null} registros com NULL")

print("\n 2️. Removendo valores invalidos (<=0)...")
linhas_antes_valor = df_clean.count()
df_clean = df_clean.filter(F.col("valor") > 0)
removidos_valor = linhas_antes_valor - df_clean.count()
print(f"Removidos {removidos_valor} registros com valor <=0")

print("\n 3. Removendo duplicatas...")
linhas_antes_dup = df_clean.count()
df_clean = df_clean.dropDuplicates(["data", "descricao", "valor", "tipo"])
removidos_dup = linhas_antes_dup - df_clean.count()
print(f"Removidos {removidos_dup} registros duplicados")

linhas_depois = df_clean.count()
print(f"\n Linhas DEPOIS da limpeza: {linhas_depois}")
print(f" Taxa de retenção: {(linhas_depois/linhas_antes)*100:.1f}%")

In [0]:
# ============================================
# 3. PADRONIZAÇÃO DE DADOS
# ============================================

print("Padronizando formato\n")

print("1️. Convertendo coluna 'data' para tipo DATE...")
df_clean = df_clean.withColumn(
    "data",
    F.to_date(F.col("data"))
)

print("2️. Padronizando descrições...")
df_clean = df_clean.withColumn(
    "descricao",
    F.upper(F.trim(F.col("descricao")))
)

print("3️. Padronizando tipo e categoria...")
df_clean = df_clean.withColumn(
    "tipo",
    F.lower(F.trim(F.col("tipo")))
).withColumn(
    "categoria",
    F.trim(F.col("categoria"))
)

print("4️. Arredondando valores para 2 casas decimais...")
df_clean = df_clean.withColumn(
    "valor",
    F.round(F.col("valor"), 2)
)

print("\n Padronização concluída")

print("\n Amostra de dados padronizados:")
display(df_clean.select(
    "data", "descricao", "valor", "tipo", "categoria"
).limit(5))

In [0]:
# ============================================
# PASSO 4: ENRIQUECIMENTO DE DADOS
# ============================================
# adiciona campos calculados uteis

print("Adicionando campos calculados...\n")

print("1️. Extraindo ano e mês...")
df_enriched = df_clean.withColumn(
    "ano",
    F.year(F.col("data"))
).withColumn(
    "mes",
    F.month(F.col("data"))
).withColumn(
    "ano_mes",
    F.date_format(F.col("data"), "yyyy-MM")
)

print("2️. Identificando dia da semana...")
df_enriched = df_enriched.withColumn(
    "dia_semana",
    F.dayofweek(F.col("data"))
).withColumn(
    "nome_dia_semana",
    F.when(F.col("dia_semana") == 1, "Domingo")
    .when(F.col("dia_semana") == 2, "Segunda")
    .when(F.col("dia_semana") == 3, "Terça")
    .when(F.col("dia_semana") == 4, "Quarta")
    .when(F.col("dia_semana") == 5, "Quinta")
    .when(F.col("dia_semana") == 6, "Sexta")
    .when(F.col("dia_semana") == 7, "Sábado")
)

print("3️. Criando valor_com_sinal para fluxo de caixa...")
df_enriched = df_enriched.withColumn(
    "valor_com_sinal",
    F.when(F.col("tipo") == "entrada", F.col("valor"))
    .otherwise(-F.col("valor"))
)

print("4️. Identificando transações de alto valor...")
df_enriched = df_enriched.withColumn(
    "alto_valor",
    F.when(F.col("valor") > 50000, True).otherwise(False)
)

print("5️. Calculando trimestre...")
df_enriched = df_enriched.withColumn(
    "trimestre",
    F.quarter(F.col("data"))
)

print("\n Enriquecimento concluído!")
print(f" Total de colunas agora: {len(df_enriched.columns)}")

print("\n Amostra com campos calculados:")
display(df_enriched.select(
    "data", "valor", "tipo", "ano_mes", 
    "nome_dia_semana", "valor_com_sinal", "alto_valor"
).limit(5))

In [0]:
# ============================================
# 5. SALVAR CAMADA SILVER
# ============================================

print("Salvando dados na camada Silver...")

# timestamp de processamento Silver
df_silver = df_enriched.withColumn(
    "silver_processed_at",
    F.current_timestamp()
)

df_silver = df_silver.select(
    "bronze_id",
    "load_timestamp",
    "silver_processed_at",
    "source_file",
    "data",
    "ano",
    "mes",
    "ano_mes",
    "trimestre",
    "dia_semana",
    "nome_dia_semana",
    "descricao",
    "valor",
    "valor_com_sinal",
    "tipo",
    "categoria",
    "alto_valor"
)

df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(TARGET_TABLE)

print(" Tabela Silver criada")

silver_count = spark.read.table(TARGET_TABLE).count()
print(f" Linhas na tabela Silver: {silver_count}")

In [0]:
#============================================
# PASSO 6: VALIDAÇÃO FINAL SILVER
# ============================================

print("Validação camada Silver\n")

df_val = spark.read.table(TARGET_TABLE)

print("1️. Verificando integridade:")
nulls_data = df_val.filter(F.col("data").isNull()).count()
nulls_valor = df_val.filter(F.col("valor").isNull()).count()
print(f"   NULLs em 'data': {nulls_data} (deve ser 0)")
print(f"   NULLs em 'valor': {nulls_valor} (deve ser 0)")

print("\n 2️. Verificando valores:")
valores_invalidos = df_val.filter(F.col("valor") <= 0).count()
print(f"   Valores <=0: {valores_invalidos} (deve ser 0)")

print("\n 3️. Verificando duplicatas:")
total = df_val.count()
distinct = df_val.select("data", "descricao", "valor").distinct().count()
duplicatas = total - distinct
print(f"   Duplicatas: {duplicatas} (deve ser 0)")

print("\n 4️. Distribuição por categoria:")
df_val.groupBy("categoria") \
    .agg(
        F.count("*").alias("quantidade"),
        F.sum("valor").alias("valor_total")
    ) \
    .orderBy("quantidade", ascending=False) \
    .show()

print("5️. Período dos dados:")
df_val.select(
    F.min("data").alias("data_inicial"),
    F.max("data").alias("data_final"),
    F.countDistinct("ano_mes").alias("meses_distintos")
).show()

print("\n Camada Silver validada e pronta para analise.")
